# Segmenting and Clustering Neighborhoods in Toronto part I
1.Create Jupiter Notebook & import libraries

In [156]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import random
import csv
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

In [324]:
print('Libraries imported.')

Libraries imported.


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [325]:
import folium

In [326]:
from bs4 import BeautifulSoup

2.Scrape Wikipedia page

In [327]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(data, 'html.parser')


In [328]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"88c1650a-329d-4919-8522-bbc7e0e91068","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [329]:
right_table=soup.find('table',class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [330]:
all_tables = soup.find_all('table')
all_tables


[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


3.Create dataframe

* The dataframe consist of three columns: Postcode, Borough, Neighborhood

In [331]:
postcode = []
borough = []
neighborhood = []

In [332]:
for row in right_table.findAll('tr'):
    
    cells= row.findAll('td')
    
    if len(cells) ==3:
        postcode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))
        
        

In [333]:
df = pd.DataFrame({'Postcode': postcode,
                       'Borough': borough,
                       'Neighborhood': neighborhood})

In [334]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


* Drop Non assigned values in 'Borough' column

In [335]:
df.drop(df[df.Borough.str.contains('Not assigned')].index,inplace =True)

In [336]:
df = df.reset_index(drop=True)

In [337]:
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


* Grouping Neighborhood

In [338]:
df.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ",".join (x))

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge\n"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park\n"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge\n"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


* Checking Non assigned value for Neighborhood

In [339]:
df.Neighborhood.str.contains('Not assigned')

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

* use the .shape method to print the number of rows of your dataframe

In [121]:
df.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto part II

* geting the latitude and the longitude coordinates of each neighborhood

In [340]:
url = 'http://cocl.us/Geospatial_data'

In [341]:
gecod = pd.read_csv(url)

In [316]:
gecod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


* useing the Geocoder package or the csv file to create the following dataframe

In [345]:
df['Latitude'] = gecod['Latitude']
df['Longitude'] = gecod['Longitude']

In [346]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


# Segmenting and Clustering Neighborhoods in Toronto part III

Explore and cluster the neighborhoods in Toronto

Useing  geopy library to get the latitude and longitude values of Toronto.

In [351]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [357]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

utilizing the Foursquare API to explore the neighborhoods and segment them

In [358]:
CLIENT_ID = 'FOG4IUBUX0UL2ZPE0YSHWNS41D03YSNNUTINAEZ5XNF2TWUO' # your Foursquare ID
CLIENT_SECRET = 'VHXKEZ4PQALIUCE4RCP13DM5SELIXT1Z0IWAVINUUWLXAIIE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: FOG4IUBUX0UL2ZPE0YSHWNS41D03YSNNUTINAEZ5XNF2TWUO
CLIENT_SECRET:VHXKEZ4PQALIUCE4RCP13DM5SELIXT1Z0IWAVINUUWLXAIIE


Geting the neighborhood's latitude and longitude values

In [370]:
def getNearbyVenues(names, latitudes, longitudes, radius=200, LIMIT=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create a new dataframe called toronto_venues

In [371]:
# type your answer here

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue, Humber Valley Village

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto, Broadview North (Old East York)

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinit

checking the size of the resulting dataframe

In [372]:
print(toronto_venues.shape)
toronto_venues.head()

(526, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
1,"Islington Avenue, Humber Valley Village",43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
2,"Garden District, Ryerson",43.692657,-79.264848,The Birchcliff,43.691666,-79.264532,Café
3,Glencairn,43.757410,-79.273304,Scarborough LRT,43.756465,-79.272194,Light Rail Station
4,St. James Town,43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant


 checking how many venues were returned for each neighborhood

In [373]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,1,1,1,1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",7,7,7,7,7,7
Bayview Village,14,14,14,14,14,14
"Bedford Park, Lawrence Manor East",8,8,8,8,8,8
"Birch Cliff, Cliffside West",27,27,27,27,27,27
"Brockton, Parkdale Village, Exhibition Place",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",2,2,2,2,2,2
Canada Post Gateway Processing Centre,4,4,4,4,4,4
Church and Wellesley,2,2,2,2,2,2


In [374]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 154 uniques categories.


Analyze Each Neighborhood

In [375]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Mac & Cheese Joint,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Islington Avenue, Humber Valley Village",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Garden District, Ryerson",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Glencairn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. James Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [378]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flower Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Mac & Cheese Joint,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoothie Shop,Soup Place,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
1,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,

confirming new size

In [379]:
toronto_grouped.shape

(59, 154)

printing each neighborhood along with the top 5 most common venues

In [380]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
                      venue  freq
0                      Café   1.0
1               Yoga Studio   0.0
2                    Museum   0.0
3  Mediterranean Restaurant   0.0
4        Mexican Restaurant   0.0


----Bathurst Manor, Wilson Heights, Downsview North
----
                       venue  freq
0  Middle Eastern Restaurant  0.14
1                Pizza Place  0.14
2                Coffee Shop  0.14
3              Deli / Bodega  0.14
4        Fried Chicken Joint  0.14


----Bayview Village
----
                 venue  freq
0    Indian Restaurant  0.14
1       Sandwich Place  0.14
2          Yoga Studio  0.07
3          Supermarket  0.07
4  Housing Development  0.07


----Bedford Park, Lawrence Manor East
----
                 venue  freq
0  Japanese Restaurant  0.25
1            Gastropub  0.12
2             Creperie  0.12
3          Coffee Shop  0.12
4           Poke Place  0.12


----Birch Cliff, Cliffside West
----
              venue  freq
0       Coffee Shop  0.

put that into a pandas dataframe

In [381]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

 creating the new dataframe and display the top 10 venues for each neighborhood.

In [383]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Café,Women's Store,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Middle Eastern Restaurant,Pizza Place,Restaurant,Deli / Bodega,Fried Chicken Joint,Sushi Restaurant,Diner,Dim Sum Restaurant,Dessert Shop
2,Bayview Village,Indian Restaurant,Sandwich Place,Yoga Studio,Supermarket,Gas Station,Housing Development,Intersection,Middle Eastern Restaurant,Pharmacy,Pizza Place
3,"Bedford Park, Lawrence Manor East",Japanese Restaurant,Coffee Shop,Gastropub,Creperie,Poke Place,Italian Restaurant,Camera Store,Women's Store,Department Store,Discount Store
4,"Birch Cliff, Cliffside West",Steakhouse,Coffee Shop,Asian Restaurant,Garden,Concert Hall,Noodle House,Opera House,Park,Pharmacy,Japanese Restaurant


Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [384]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 1, 1, 1, 1, 1, 4, 1], dtype=int32)

 createing a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [389]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.806686,-79.194353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.784535,-79.160497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,1.0,1.0,Korean Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Women's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 visualizing the resulting clusters

In [394]:
toronto_merged.dropna(inplace =True)


In [395]:
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917,1.0,1.0,Korean Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Women's Store
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.744734,-79.239476,1.0,1.0,Playground,Women's Store,Cupcake Shop,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,2.0,2.0,Café,Women's Store,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
10,M6B,North York,Glencairn,43.757410,-79.273304,1.0,1.0,Light Rail Station,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Women's Store
15,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,1.0,1.0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Bank,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Supermarket,Noodle House


In [398]:
toronto_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 3 to 101
Data columns (total 17 columns):
Postcode                  62 non-null object
Borough                   62 non-null object
Neighborhood              62 non-null object
Latitude                  62 non-null float64
Longitude                 62 non-null float64
Cluster_Labels            62 non-null float64
Cluster Labels            62 non-null float64
1st Most Common Venue     62 non-null object
2nd Most Common Venue     62 non-null object
3rd Most Common Venue     62 non-null object
4th Most Common Venue     62 non-null object
5th Most Common Venue     62 non-null object
6th Most Common Venue     62 non-null object
7th Most Common Venue     62 non-null object
8th Most Common Venue     62 non-null object
9th Most Common Venue     62 non-null object
10th Most Common Venue    62 non-null object
dtypes: float64(4), object(13)
memory usage: 8.7+ KB


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 Examine Clusters

Cluster 1

In [401]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,0.0,0.0,Performing Arts Venue,Deli / Bodega,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Cupcake Shop


Cluster 2

In [405]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1.0,1.0,Korean Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Women's Store
5,Etobicoke,1.0,1.0,Playground,Women's Store,Cupcake Shop,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
10,North York,1.0,1.0,Light Rail Station,Department Store,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Women's Store
15,Downtown Toronto,1.0,1.0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Bank,Coffee Shop,Pharmacy,Pizza Place,Sandwich Place,Supermarket,Noodle House
17,Etobicoke,1.0,1.0,Golf Course,Women's Store,Cupcake Shop,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
18,Scarborough,1.0,1.0,Clothing Store,Women's Store,Coffee Shop,Bank,Movie Theater,Mobile Phone Shop,Pharmacy,Chocolate Shop,Liquor Store,Kids Store
27,North York,1.0,1.0,Restaurant,Asian Restaurant,Shopping Mall,Dim Sum Restaurant,Gym,Discount Store,Japanese Restaurant,Bike Shop,Italian Restaurant,Chinese Restaurant
28,North York,1.0,1.0,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Restaurant,Deli / Bodega,Fried Chicken Joint,Sushi Restaurant,Diner,Dim Sum Restaurant,Dessert Shop
31,West Toronto,1.0,1.0,Wings Joint,Home Service,Deli / Bodega,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
34,North York,1.0,1.0,French Restaurant,Intersection,Women's Store,Farmers Market,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


Cluster 3

In [414]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,2.0,2.0,Café,Women's Store,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
78,Scarborough,2.0,2.0,Café,Women's Store,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


Cluster 4

In [415]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,East York,3.0,3.0,Park,Convenience Store,Women's Store,Department Store,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
37,West Toronto,3.0,3.0,Park,Trail,Cupcake Shop,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
42,Downtown Toronto,3.0,3.0,Park,Fish & Chips Shop,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
48,Downtown Toronto,3.0,3.0,Park,Women's Store,Deli / Bodega,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store


Cluster 5

In [416]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Central Toronto,4.0,4.0,Coffee Shop,Women's Store,Deli / Bodega,Electronics Store,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
89,Etobicoke,4.0,4.0,Coffee Shop,Pharmacy,Pizza Place,Cupcake Shop,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
96,Downtown Toronto,4.0,4.0,Pizza Place,Women's Store,Cupcake Shop,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
99,Downtown Toronto,4.0,4.0,Coffee Shop,Pizza Place,Women's Store,Deli / Bodega,Dumpling Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store


Obserwation
On the base above analysis we can see that 526 venues return for each nighborhoods. The larges areas are: Downsview 55 venues 10% of tolal, Lowrence Park 45 venues 8.5% of tolal , Westmount 35 venues 6.6% of total.
Data contains 154 unik categories, looking at frequensy of visitors Leasge venue Park has the highest resoult 0.67, Humberlea, Emery venue Coffe Shop 0.38, University of Toronto, Harbord venue Museum 0.33

Cluster examination
Cluster2 is the bigest w hile Cluster one has only one neighborhood 